# Generalized Standard Material Model API Demonstration

This notebook demonstrates the usage of the `GSMModel` class, which provides a bridge between 
symbolic material definitions (using `GSMDef`) and executable numerical models with concrete parameter values.

We'll use the elastic-damage model (`GSM1D_ED`) as an example to show how simple it is to create, parametrize, and 
visualize material responses using this framework.


In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp
import traits.api as tr
import bmcs_utils.api as bu

# Import the GSM framework
from bmcs_matmod.gsm_lagrange.gsm_model import GSMModel

# Import the specific model we'll use for demonstration
from bmcs_matmod.gsm_lagrange.gsm1d_ed import GSM1D_ED

# For nicer plot display
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12
plt.rcParams['lines.linewidth'] = 2
try:
    plt.style.use('bmcs')
except:
    pass

# For LaTeX rendering in plots
plt.rcParams['text.usetex'] = False


## 1. Examining the Elastic-Damage Model

First, let's take a look at the `GSM1D_ED` class to understand its structure. This is a 1D elastic damage model implemented using the GSM framework.

In [ ]:
# Examine the GSM1D_ED class
gsm1d_ed = GSM1D_ED()
gsm1d_ed.print_potentials()

## 2. Creating a Material Model from GSM1D_ED

Now, let's use the `GSMModel` class to create an executable model with specific parameter values. The `GSMModel` class automatically:

1. Analyzes the symbolic model structure
2. Creates traits for all parameters  
3. Provides methods for numerical simulation

This allows us to work with material models in a more intuitive way, defining parameter values directly and running simulations without managing the symbolic-to-numerical conversion manually.


In [ ]:
# Create a material model from the GSM1D_ED symbolic model
ed_material = GSMModel(GSM1D_ED)

# View the default parameters
print("Default parameters:")
for param_sym, name in ed_material.trait_model_params.items():
    value = getattr(ed_material, name)
    print(f"  {name} = {value} ({param_sym})")

print("\n" + "-"*50 + "\n")

# Set parameters to customize the model using set_params
ed_material.set_params(
    E=20000.0,    # Young's modulus (MPa)
    S=1,          # Damage threshold strain
    c=1,          # ratio
    eps_0=0.0     # Initial inelastic strain
)

# View the updated parameters
print("Updated parameters:")
for param_sym, name in ed_material.trait_model_params.items():
    value = getattr(ed_material, name)
    print(f"  {name} = {value} ({param_sym})")

print("\n" + "-"*50 + "\n")

## 3. Monotonic Tension Test

Let's simulate a monotonic tension test to see how our material behaves. This will demonstrate the strain-softening behavior characteristic of damage mechanics.

In [ ]:
# Define a monotonic tensile strain history
n_steps = 1000
strain_max = 0.13  # Maximum strain
strain = np.linspace(0, strain_max, n_steps)
time = np.linspace(0, 1.0, n_steps)

# Run the simulation with our material model
# Using the standard get_F_response method which handles the correct parameter preparation
response = ed_material.get_F_response(strain, time)

# Unpack the results
t_t, eps_ta, sig_ta, Eps_t, Sig_t, iter_t, lam_t, (d_t_t, d_eps_ta) = response

# Extract and reshape for easier plotting
eps = eps_ta[:, 0]
sig = sig_ta[:, 0]
omega = Eps_t[:, 0, 0]  # First internal variable (damage variable)

In [ ]:
Eps_t.shape

In [ ]:
# Plot the stress-strain curve
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Stress-strain curve
ax1.plot(eps, sig, 'b-', linewidth=2)
ax1.set_xlabel('Strain $\\varepsilon$')
ax1.set_ylabel('Stress $\\sigma$ (MPa)')
ax1.set_title('Stress-Strain Curve (Tensile Test)')
ax1.grid(True)

# Evolution of internal variable (kappa - damage)
ax2.plot(eps, omega, 'g-', linewidth=2)
ax2.set_xlabel('Strain $\\varepsilon$')
ax2.set_ylabel('Internal variable $\\kappa$')
ax2.set_title('Evolution of Damage Variable')
ax2.grid(True)

plt.tight_layout()